In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read(filename, path='dataset/'):
    import numpy as np
    return np.loadtxt(filename,  delimiter=',')

def center(data):
    '''
        center data and return mean
    '''
    import numpy as np
    mean = np.mean(data[:, 2])
    data[:, 2] = data[:, 2] - mean
    return mean

def data_to_list(N, M, data):
    '''
        data -> user, item, score
        return list of items rated by user &
               list of users rated item.
    '''
    user_list = [[] for _ in range(N+1)]
    item_list = [[] for _ in range(M+1)]
    for i in range(data.shape[0]):
        u_id = int(data[i][0])
        i_id = int(data[i][1])
        score = data[i][2]
        user_list[u_id].append([i_id, score])
        item_list[i_id].append([u_id, score])
    return user_list, item_list

def plot(x, y, xlabel, ylabel, title, color):
    import matplotlib.pyplot as plt
    plt.plot(x, y, color=color)
    plt.plot(x, y, color+'o')
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()

def edge_list_to_adj_list(MAX_ID, edge_list, LIMIT=-1):
    '''
        MAX_ID -> maximum id of users
        LIMIT -> maximum outdegree
                 for default value there is no limit
        return G, G^T
    '''
    import numpy as np
    adj = [[] for _ in range(MAX_ID+1)]
    adjT = [[] for _ in range(MAX_ID+1)]
    for l in edge_list:
        [u, v] = l[0],l[1]
        u, v = int(u), int(v)
        if max(u, v) <= MAX_ID and (LIMIT == -1 or len(adj[u]) < LIMIT):
            adj[u].append(v)
            adjT[v].append(u)
    return adj, adjT

In [ ]:
import numpy as np
from numpy.linalg import inv
import pickle
import os
from sklearn.metrics import log_loss
import torch



class Model:
    def __init__(self, train_data=None, network=None, network_T=None, D=1, sigma=1,
                sigma_u=1, sigma_v=1, sigma_w=1, iterations=10, load=True):
        if load:
            self.mean_U = np.load('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/E_U.npy')
            self.mean_V = np.load('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/E_V.npy')
            self.D = self.mean_U.shape[1]
            self.network = network
            with open('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/E_W.txt', 'rb') as fp:
                self.mean_W = pickle.load(fp)
                return
        self.sigma = sigma
        self.sigma_u = sigma_u
        self.sigma_v = sigma_v
        self.sigma_w = sigma_w
        self.iterations = iterations
        self.network = network
        self.network_T = network_T
        self.D = D
        self.N, self.M, _ = np.max(train_data, axis=0)
        self.N, self.M = int(self.N), int(self.M)
        self.train_data = train_data
        self.user_list, self.item_list = data_to_list(self.N, self.M, self.train_data)

    def init(self):
        '''
            initialized mean and cov of
            Qs uniformly in [0, 1].
        '''
        self.mean_U = np.random.rand(self.N+1, self.D)
        self.cov_U = np.random.rand(self.N+1, self.D, self.D)
        self.mean_V = np.random.rand(self.M+1, self.D)
        self.cov_V = np.random.rand(self.M+1, self.D, self.D)
        self.mean_W = [[np.random.rand() for i1 in range(len(self.network[i]))] for i in range(self.N+1)]
        self.var_W = [[np.random.rand() for i1 in range(len(self.network[i]))] for i in range(self.N+1)]

    def test(self, data):
        '''
            return RMSE of model on validation data
        '''
        bce = 0.
        mae = 0.
        for i, j, score in data:
            i, j = int(i), int(j)
            pred = (self.E_U(i).T.dot(self.E_V(j)))[0,0]
            for i1 in self.network[i]:
                pred += (self.E_W(i, i1) * self.E_U(i1).dot(self.E_V(j)))[0,0]
            if score == 1:
              bce += -np.log(torch.sigmoid(torch.FloatTensor([pred]))).item()
            else:
              bce += -np.log(1 - torch.sigmoid(torch.FloatTensor([pred]))).item()
            mae += abs(pred - score)
        bce /= data.shape[0]
        mae /= data.shape[0]
        #rmse = np.sqrt(rmse)
        return bce,mae

    def predict(self, data):
        '''
            return prediction for given data.
            data -> 2d array (user, item)
        '''
        preds = []
        for i, j in data:
            i, j = int(i), int(j)
            pred = (self.E_U(i).T.dot(self.E_V(j)))[0,0]
            for i1 in self.network[i]:
                pred += (self.E_W(i, i1) * self.E_U(i1).dot(self.E_V(j)))[0,0]
            preds.append(pred)
        return np.array(pred)

    def train(self):
        self.init()
        bce,mae = self.test(self.train_data)
        bce_list = [bce]
        mae_list = [mae]
        print('iteration', 0, ' BCE:', bce)
        print('iteration', 0, ' MAE:', mae)
        for k in range(1, self.iterations+1):
            for i in range(1, self.N+1):
                for i1 in self.network[i]:
                    self.update_W(i, i1)
            for i in range(1, self.N+1):
                self.update_U(i)
            for j in range(1, self.M+1):
                self.update_V(j)
            bce,mae = self.test(self.train_data)
            print('iteration', k, ' BCE:', bce)
            print('iteration', k, ' MAE:', mae)
            bce_list.append(bce)
            mae_list.append(mae)
        if not os.path.exists('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/'):
            os.makedirs('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/')
        np.save('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/E_U.npy', self.mean_U)
        np.save('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/E_V.npy', self.mean_V)
        with open('/content/drive/MyDrive/Implicit/Ciao/Models/SocialMF/E_W.txt', 'wb') as fp:
            pickle.dump(self.mean_W, fp)
        return bce_list

    def update_U(self, i):
        '''
            update mean and cov of Q(Ui)
        '''
        cov_inv = np.eye(self.D) / self.sigma_u
        mean = 0
        for j, score in self.user_list[i]:
            E_VVJ = self.E_VV(j)
            cov_inv += E_VVJ / self.sigma
            mean += (score * self.E_V(j) - E_VVJ.dot(self.E_S(i).T)) / self.sigma

        for i2 in self.network_T[i]:
            for j, score in self.user_list[i2]:
                E_VVJ = self.E_VV(j)
                cov_inv += (self.E_WW(i2, i) * E_VVJ) / self.sigma
                mean += self.E_W(i2, i) * \
                    (score * self.E_V(j) - E_VVJ.dot(self.E_U(i2) + self.E_S_(i2, i).T))
        cov = inv(cov_inv)
        mean = cov.dot(mean)
        self.mean_U[i, :] = mean
        self.cov_U[i, :, :] = cov

    def update_V(self, j):
        '''
            update mean and cov of Q(Vj)
        '''
        cov_inv = np.eye(self.D) / self.sigma_v
        mean = 0
        for i, score in self.item_list[j]:
            E_Si = self.E_S(i)
            cov_inv += (self.E_UU(i)+ 2 * self.E_U(i).dot(E_Si) +
                        self.E_SS(i)) / self.sigma
            mean += score * (self.E_U(i) + E_Si.T) / self.sigma
        cov = inv(cov_inv)
        mean = cov.dot(mean)
        self.mean_V[j, :] = mean
        self.cov_V[j, :, :] = cov

    def update_W(self, i, i1):
        '''
            update mean and cov of Q(Wii')
        '''
        rho = 1. / self.sigma_w
        mean = 0
        for j, score in self.user_list[i]:
            EUi1 = self.E_U(i1)
            EUi = self.E_U(i)
            E_VVJ = self.E_VV(j)
            rho += (self.E_UVVU(i1, j)) / self.sigma
            mean += (EUi1.T.dot(self.E_V(j)) * score -
                    EUi.T.dot(E_VVJ.dot(self.E_U(i1))) -
                    EUi1.T.dot(E_VVJ.dot(self.E_S_(i, i1)))) / self.sigma

        var = 1. / rho
        mean = var * mean
        idx = self.network[i].index(i1)
        self.mean_W[i][idx] = mean
        self.var_W[i][idx] = var

    def E_U(self, i):
        '''
            return EQ[Ui]
        '''
        return self.mean_U[i].reshape((self.D, 1))

    def E_UU(self, i):
        '''
            return EQ[UiUi^T]
        '''
        m_u_i = self.mean_U[i].reshape((self.D, 1))
        return (self.cov_U[i, :, :] + m_u_i.dot(m_u_i.T))

    def E_V(self, j):
        '''
            return EQ[Vj]
        '''
        return self.mean_V[j].reshape((self.D, 1))

    def E_VV(self, j):
        '''
            return E[VjVj^T]
        '''
        m_v_j = self.mean_V[j].reshape((self.D, 1))
        return (self.cov_V[j, :, :] + m_v_j.dot(m_v_j.T))

    def E_W(self, i, i1):
        '''
            return EQ[Wii']
        '''
        idx = self.network[i].index(i1)
        return self.mean_W[i][idx]

    def E_WW(self, i, i1):
        '''
            return EQ[Wii'Wii'^T]
        '''
        idx = self.network[i].index(i1)
        return self.var_W[i][idx] + self.mean_W[i][idx]**2

    def E_S(self, i):
        '''
            return EQ[Si]
        '''
        ret = np.zeros((self.D, 1))
        for i1 in self.network[i]:
            ret += self.E_W(i, i1) * self.E_U(i1)
        return ret

    def E_S_(self, i, k):
        '''
            return EQ[Si^(-k)]
        '''
        return self.E_S(i) - self.E_W(i, k) * self.E_U(k)

    def E_SS(self, i):
        '''
            return EQ[Si^TSi]
        '''
        ret = 0
        for i1 in self.network[i]:
            ret += self.E_WW(i, i1) * self.E_UU(i1)
            for i2 in self.network[i]:
                if i1 != i2:
                    ret += self.E_W(i, i1) * self.E_W(i, i2) * self.E_U(i1) * self.E_U(i2).T
        return ret

    def E_UVVU(self, i, j):
        '''
            return EQ[Ui^TVjVj^TUi]
        '''
        e = np.ones((self.D, 1))
        tmp1 = e.T.dot(self.E_UU(i).dot(e))
        tmp2 = e.T.dot(self.E_VV(j).dot(e))
        return tmp1 * tmp2

In [ ]:
import pandas as pd

train_data = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/trainImplicit.csv")[["userId","productId","rating"]].values
valid_data = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/ratings_data_test_implicit_2_0.csv")[["userId","productId","rating"]].values
edge_list = pd.read_csv('/content/drive/MyDrive/Implicit/Ciao/implicitTrust.csv')[["from","to","value"]].values

#train_data = pd.read_csv("/content/drive/MyDrive/Epinions/Ciao/ratings_data_train.csv")[["userId","productId","rating"]].values
#valid_data = pd.read_csv("/content/drive/MyDrive/Epinions/Ciao/ratings_data_test_2_0.csv")[["userId","productId","rating"]].values
#edge_list = pd.read_csv('/content/drive/MyDrive/Epinions/Ciao/trust_final.csv')[["from","to","value"]].values


MAX_ID = 7373
LIMIT = 1

network, network_transpose = edge_list_to_adj_list(MAX_ID, edge_list, LIMIT)

#train_mean = center(train_data)
#valid_data[:, 2] = valid_data[:, 2] - train_mean

# ----------------------------------
#         train Model
# ----------------------------------
sigma = 1
sigma_u = 1
sigma_v = 1
sigma_w = 1
D = 1
iterations = 30

model = Model(train_data, network, network_transpose, D, sigma, sigma_u,
                    sigma_v, sigma_w, iterations)
'''bce_list = model.train()

x = [i for i in range(1, iterations+1) if i % 2 == 0 or (i == 1)]
y = [bce_list[i] for i in range(1, iterations+1) if i % 2 == 0 or (i == 1)]
bce,mae = model.test(valid_data)
print("BCE on validation data: ",bce,"MAE on validation data: ",mae)'''

'bce_list = model.train()\n\nx = [i for i in range(1, iterations+1) if i % 2 == 0 or (i == 1)]\ny = [bce_list[i] for i in range(1, iterations+1) if i % 2 == 0 or (i == 1)]\nbce,mae = model.test(valid_data)\nprint("BCE on validation data: ",bce,"MAE on validation data: ",mae)'

In [ ]:
import os
import math
import warnings
import random
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return len(test_in_top_k) * 1.0 / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()

#path = "/content/drive/MyDrive/Epinions/testData_Final/"
#path = "/content/drive/MyDrive/Epinions/Ciao/"
path = "/content/drive/MyDrive/Implicit/Ciao/testImplicit.csv"
#files = os.listdir(path)
#model.eval()
c=1
test = pd.read_csv(path)
test = test[["userId","productId","rating"]]
test_negatives = _sample_negative(list(test["productId"].unique()),test)
a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
del test_negatives
test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
for row in a.itertuples():
  test_users.append(int(row.userId))
  test_items.append(int(row.productId))
  ratings.append(float(row.rating))
  for i in range(len(row.negative_samples)):
      negative_users.append(int(row.userId))
      negative_items.append(int(row.negative_samples[i]))
      negative_ratings.append(float(0))  # negative samples get 0 rating
a = np.vstack([test_users,test_items]).transpose()
preds = []
for i, j in a:
  i, j = int(i), int(j)
  pred = (model.E_U(i).T.dot(model.E_V(j)))[0,0]
  for i1 in model.network[i]:
      pred += (model.E_W(i, i1) * model.E_U(i1).dot(model.E_V(j)))[0,0]
  preds.append(pred)

predr = np.array(preds)


b = np.vstack([negative_users,negative_items]).transpose()
preds_neg = []
for i, j in b:
  i, j = int(i), int(j)
  pred = (model.E_U(i).T.dot(model.E_V(j)))[0,0]
  for i1 in model.network[i]:
      pred += (model.E_W(i, i1) * model.E_U(i1).dot(model.E_V(j)))[0,0]
  preds_neg.append(pred)

neg_predr = np.array(preds_neg)
subjects = [test_users,test_items,predr,negative_users,negative_items,neg_predr]
test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
# the golden set
test = pd.DataFrame({'user': test_users,
                    'test_item': test_items,
                    'test_score': test_scores})
# the full set
full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                  'item':  np.append(neg_items,test_items),
                  'score':  np.append(neg_scores,test_scores)})
full = pd.merge(full, test, on=['user'], how='left')
# rank the items according to the scores for each user
full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
full.sort_values(['user', 'rank'], inplace=True)
subjects = full
print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))

Hit Ratio is:  0.36
NDCG Value is:  0.23


In [ ]:
top_k = full[full['rank']<=5]
top_k

user  item     score  test_item  test_score  rank
53340572     0   576  0.730134        259    0.605033   1.0
53340573     0   576  0.730134        622    0.365805   2.0
53340574     0   576  0.730134        493    0.000000   3.0
53340575     0   576  0.730134        398    0.272765   4.0
53340576     0   576  0.730134        422    0.683881   5.0
...        ...   ...       ...        ...         ...   ...
48974063  7372  6646  0.931459      22748    0.796693   1.0
48974064  7372  6646  0.931459      22441    0.921338   2.0
48974065  7372  6646  0.931459      63566    0.877281   3.0
48974183  7372  6646  0.931459      22748    0.796693   4.0
48974184  7372  6646  0.931459      22441    0.921338   5.0

[33565 rows x 6 columns]

In [ ]:
train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Ciao/train_popularity.txt")
x = top_k.groupby("user")
users = list(x.groups.keys())
pops = []
for i in users:
  a = x.get_group(i)
  items = a['item'].values
  for j in items:
    s= train_popularity[j]
    pops.append(s/5)
print(sum(pops)/len(pops))

12.13626098614603


In [ ]:
'''
ratings_data_test_implicit_2_0.csv
Hit Ratio is:  0.07
NDCG Value is:  0.05
ratings_data_test_implicit_3_0.csv
Hit Ratio is:  0.12
NDCG Value is:  0.08
ratings_data_test_implicit_4_0.csv
Hit Ratio is:  0.22
NDCG Value is:  0.14
ratings_data_test_implicit_5_0.csv
Hit Ratio is:  0.33
NDCG Value is:  0.22
ratings_data_test_implicit_6_0.csv
Hit Ratio is:  0.52
NDCG Value is:  0.33
ratings_data_test_implicit_7_0.csv
Hit Ratio is:  0.86
NDCG Value is:  0.59
ratings_data_test_implicit_8_0.csv
Hit Ratio is:  1.14
NDCG Value is:  0.92
ratings_data_test_implicit_9_0.csv
Hit Ratio is:  1.18
NDCG Value is:  0.97
ratings_data_test_implicit_10_0.csv
Hit Ratio is:  1.23
NDCG Value is:  1.02
'''

In [ ]:
import os
import math
import warnings
import random
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return len(test_in_top_k) * 1.0 / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()

#path = "/content/drive/MyDrive/Epinions/testData_Final/"
#path = "/content/drive/MyDrive/Epinions/Ciao/"
path = "/content/drive/MyDrive/Implicit/Ciao/"
files = os.listdir(path)
#model.eval()
c=1
for f in files:
  if("test" in f and f in ["ratings_data_test_implicit1_1_0.csv","ratings_data_test_implicit1_2_0.csv","ratings_data_test_implicit1_5_0.csv","ratings_data_test_implicit1_10_0.csv"]):
    print(f)
    test = pd.read_csv(path+f)
    test = test[["userId","productId","rating"]]
    test_negatives = _sample_negative(list(test["productId"].unique()),test)
    a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
    del test_negatives
    test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
    for row in a.itertuples():
      test_users.append(int(row.userId))
      test_items.append(int(row.productId))
      ratings.append(float(row.rating))
      for i in range(len(row.negative_samples)):
          negative_users.append(int(row.userId))
          negative_items.append(int(row.negative_samples[i]))
          negative_ratings.append(float(0))  # negative samples get 0 rating
    a = np.vstack([test_users,test_items]).transpose()
    preds = []
    for i, j in a:
      i, j = int(i), int(j)
      pred = (model.E_U(i).T.dot(model.E_V(j)))[0,0]
      for i1 in model.network[i]:
          pred += (model.E_W(i, i1) * model.E_U(i1).dot(model.E_V(j)))[0,0]
      preds.append(pred)

    predr = np.array(preds)


    b = np.vstack([negative_users,negative_items]).transpose()
    preds_neg = []
    for i, j in b:
      i, j = int(i), int(j)
      pred = (model.E_U(i).T.dot(model.E_V(j)))[0,0]
      for i1 in model.network[i]:
          pred += (model.E_W(i, i1) * model.E_U(i1).dot(model.E_V(j)))[0,0]
      preds_neg.append(pred)

    neg_predr = np.array(preds_neg)
    subjects = [test_users,test_items,predr,negative_users,negative_items,neg_predr]
    test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
    neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
    # the golden set
    test = pd.DataFrame({'user': test_users,
                        'test_item': test_items,
                        'test_score': test_scores})
    # the full set
    full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                      'item':  np.append(neg_items,test_items),
                      'score':  np.append(neg_scores,test_scores)})
    full = pd.merge(full, test, on=['user'], how='left')
    # rank the items according to the scores for each user
    full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
    full.sort_values(['user', 'rank'], inplace=True)
    subjects = full
    print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
    print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))

ratings_data_test_implicit1_2_0.csv
Hit Ratio is:  0.07
NDCG Value is:  0.03
ratings_data_test_implicit1_5_0.csv
Hit Ratio is:  0.97
NDCG Value is:  0.74
ratings_data_test_implicit1_10_0.csv
Hit Ratio is:  1.06
NDCG Value is:  1.02
ratings_data_test_implicit1_1_0.csv
Hit Ratio is:  0.04
NDCG Value is:  0.02
